In [ ]:
import requests
import lxml.html as html
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from fake_useragent import UserAgent
import math

Url principal y creación del user-agent aleatorio

In [ ]:
# Url principal
urlPrincipal = 'https://www.dieselogasolina.com/buscador-gasolineras.html'

# Item para generación aleatoria de user-agent
ua = UserAgent()

Listado expresiones X_PATH

In [ ]:
# Url de todas las provincias
urlProvincias = '//div[@class="mas_provincias"]/ul/li/a[starts-with(@title, "Gasolineras en la provincia")]/@href'

# Url de la información de las gasolineras
urlGasolinerasX = '//a[@title="Ver datos de esta gasolinera y precios"]'

# Boton aceptar cookies
acceptCookieX = '//*[@id="privacy_block"]/div[3]/p/button'

# Número de paginas de la tabla y boton de next
numPagsX = '//span/a'
nextButtonX = '//a[@class="paginate_button next"]'


# Datos de las gasolineras
localidadX = 'body/div[2]/div[2]/div[2]/div/div[1]/div[1]/div/h1/text()'
direccionX = '//h1/span/text()'
tiposGasolinaX = '//li/span'
preciosX = '//div/div[1]/div/div/div[7]/div/table/tbody/tr/td[1]'
fechaReviX = '//*[@id="localizacion"]/div[5]/div[3]/p[1]/b'
fechaUltX = '//*[@id="localizacion"]/div[5]/div[3]/p[2]/b'
datosTablaX = '//*[@id="empresa"]/div/p'

Funciones

In [ ]:
# Función para la obtención de los url de las provincias
def get_provincias(url_padre):
    userAgent = ua.random
    print("User Agent: " + userAgent)
    headers = {"User-Agent": userAgent}
    tries = range(3)
    
    for _ in tries:
        try:
            r = requests.get(url_padre, headers=headers)
        except requests.exceptions.ConnectionError:
            print("CONNECTION ERROR!")
            delay(20)
        except requests.exceptions.Timeout:
            print("TIMEOUT ERROR!")
            delay(20)
        except requests.exceptions.RequestException:
            print("REQUEST EXCEPTION ERROR!")
            delay(20)
            
    home = r.content

    parser = html.fromstring(home)

    provincias_url = parser.xpath(urlProvincias)
    
    return provincias_url

In [ ]:
# Función para extraer las urls de todas las gasolineras 
# de la url de la provincia introducida.
def get_gasolineras(provincias):
    t0 = time.time()

    urls = []
    
    for url in provincias:       
        urls.append(get_gasolineras_provincia(url)) 
        delay(20)
    
    listaPlana = []
    for elementos in urls:
        for elemento in elementos:
            listaPlana.append(elemento)
            
    conversion(time.time() - t0)
    
    return listaPlana

In [ ]:
# Función auxiliar de get_gasolineras
def get_gasolineras_provincia(url):
    driver = driver_configuration()
    listaDatos = []
    listaPlana = []

    driver.get(url)
    print("User Agent: " + driver.execute_script("return navigator.userAgent;"))
    
    WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable(
        (By.XPATH, 
        acceptCookieX))).click()
    
    numPags = [my_elem.text for my_elem in WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((
            By.XPATH, 
            numPagsX)))]
    
    numPags = int(numPags[-1])

    listaAux = [my_elem.get_attribute("href") 
             for my_elem in WebDriverWait(driver, 5).
             until(EC.presence_of_all_elements_located((
                 By.XPATH, 
                 urlGasolinerasX)))]

    listaDatos.append(listaAux)

    for i in range(numPags-1):   
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable(
                (By.XPATH, 
                nextButtonX))).click()

        listaAux = [my_elem.get_attribute("href") 
             for my_elem in WebDriverWait(driver, 5).
             until(EC.presence_of_all_elements_located((
                 By.XPATH, 
                 urlGasolinerasX)))]

        listaDatos.append(listaAux)
    
    driver.quit()

    for elementos in listaDatos:
        for elemento in elementos:
            listaPlana.append(elemento)
        
    return listaPlana

In [ ]:
# Función para explorar la url y extraer los datos de interes
def parser_content(url):
    userAgent = ua.random
    print("User Agent: " + userAgent)
    headers = {"User-Agent": userAgent}
    nTries = range(3)

    for _ in nTries:
        try:
            r = requests.get(url, headers=headers, timeout = 10)
            break
        except requests.exceptions.ConnectionError:
            print("CONNECTION ERROR!")
            delay(20)    
        except requests.exceptions.Timeout:
            print("TIMEOUT ERROR!")
            delay(20)
        except requests.exceptions.RequestException:
            print("REQUESTEXCEPTION ERROR!")
            delay(20)

    home = r.content

    parser = html.fromstring(home)

    provincia = None
    localidad = None
    direccion = None
    margen = None
    horario = None
    latitud = None
    longitud = None
    empresa = None
    precios = None
    fechaRevi = None
    fechaUlt = None
    ventPubli = None

    if r.status_code == 200:
        locProv = parser.xpath(localidadX)
        locProv = locProv[0].split(' en ')
        locProv = locProv[1].rsplit(" (", 1)
        localidad = locProv[0]
        provincia = locProv[1].replace(")", "")

        direccion = parser.xpath(direccionX)[0].strip()
        
        
            
        datosTabla = parser.xpath(datosTablaX) 
        for dato in datosTabla:
            if "Gasolinera" in dato.text_content():
                empresa = dato.text_content().replace("Gasolinera ", "").strip()
            if "Horario" in dato.text_content():
                horario = dato.text_content().replace("Horario: ", "").strip()
            if "blico" in dato.text_content().lower():
                ventPubli = dato.text_content().strip()
            if "Margen" in dato.text_content():
                margen = dato.text_content().replace("Margen de la carretera: ", "").strip()
            if "Lat,Lng" in dato.text_content():
                latLong = dato.text_content().replace("Lat,Lng: ", "").split(",")
                latitud = latLong[0].strip()
                longitud = latLong[1].strip()

        tiposGasolina = parser.xpath(tiposGasolinaX)
        precios = parser.xpath(preciosX)
        precios = list(map(lambda t, p: t.text_content() + " " + p.text_content(), 
                      tiposGasolina, 
                      precios))

        fechaRevi = parser.xpath(fechaReviX)[0].text_content().replace(
            'Precios revisados el ', "").replace(
            ".", "").strip()

        fechaUlt = parser.xpath(fechaUltX)[0].text_content().replace(".", "").strip()

    return [provincia, localidad, direccion, 
            margen, horario, latitud, longitud, empresa, 
            precios, fechaRevi, fechaUlt, ventPubli]
 

In [ ]:
# Función que configura el driver
def driver_configuration():
    # Opciones de navegación
    options = Options()
    options.add_argument('--start-maxima')
    options.add_argument('--disable-extensions')
    options.add_argument("user-agent=" + ua.random)

    return webdriver.Chrome(options=options)

In [ ]:
# Función de espera
def delay(s):
    time.sleep(s)

In [ ]:
# Función para convertir el tiempo de segundos a
# 'h m s'
def conversion(tt):
    m, h = math.modf(tt/3600)
    s, m = math.modf(m * 60)
    s = s * 60
    print(f'Tiempo total: {int(h)}h {int(m)}m {int(s)}s')

Ejecución del programa de Web-Scraping

In [ ]:
linksProvincias = get_provincias(urlPrincipal)

In [ ]:
gasolineras = get_gasolineras(linksProvincias)

In [ ]:
datos = []
num = len(gasolineras)  
ts = time.time()

# Mediante el bucle se registran los datos de cada gasolinera
for link in gasolineras:
    num -= 1
    datos.append(parser_content(link))
    print(f'Gasolineras que faltan por explorar: {num}\n')
    if num%500 == 0:
        delay(30)

conversion(time.time() - ts)

Creación del archivo CSV donde se guardarán los datos.

In [ ]:
# Creación de un dataframe con los nombres de las columnas
df = pd.DataFrame(columns=['provincia', 'localidad', 'direccion', 
            'margen', 'horario', 'latitud', 'longitud', 'empresa', 
            'precios', 'fechaRevision', 'fechaUltima', 'ventaPublico'])
print(df)

In [ ]:
# Carga de los datos en el dataframe
for info in datos:
    df=df.append({'provincia': info[0], 'localidad': info[1], 'direccion': info[2], 
                'margen': info[3], 'horario': info[4], 'latitud': info[5], 
                'longitud': info[6], 'empresa': info[7], 'precios': info[8],
                'fechaRevision': info[9], 'fechaUltima': info[10], 'ventaPublico': info[11]},
                 ignore_index=True)

In [ ]:
# Creación del archivo Gasolineras.csv
df.to_csv('Gasolineras.csv', index=False)